In [9]:
import psycopg2
import time
import random

# root id: postgres, password: 1234
# DB dbms_final owner: postgres (only user can create table) 
conn_string = "host='140.112.12.188' user='postgres' dbname='dbms_final' password='1234' port='5432'"
conn = psycopg2.connect(conn_string)
print("Connection established")

Connection established


In [10]:
import csv
csv_file = 'patient.csv'
patient_site_map = {}

with open(csv_file, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        id_number = row['ID_Number']
        site_id = row['Site_ID']
        patient_site_map[id_number] = site_id

In [ ]:
cursor = conn.cursor()

def query_patient(patient_id, doctor_site_id):
    cursor.execute(f"SET app.current_doctor_site_id = '{doctor_site_id}';")
    start_time = time.time()
    cursor.execute("SELECT * FROM Patient WHERE ID_Number = %s AND Site_ID = %s", (patient_id, doctor_site_id))
    result = cursor.fetchall()
    end_time = time.time()
    return end_time - start_time, result

cursor.execute("DROP POLICY patient_policy ON Patient;")

cursor.execute("ALTER TABLE Patient ENABLE ROW LEVEL SECURITY;")

cursor.execute("""
    CREATE POLICY patient_policy ON Patient
    USING (EXISTS (
        SELECT 1 FROM Doctor 
        WHERE Doctor.Site_ID = Patient.Site_ID 
        AND Doctor.ID_Number = current_setting('app.current_doctor_id')
    ));
""")

query_times_same_site = []
query_times_diff_site = []

for _ in range(100): 
    print(f"The {_}st test.")
    for patient_id, patient_site_id in patient_site_map.items():
        doctor_site_id = random.choice(range(5))
        query_time, _ = query_patient(patient_id, doctor_site_id)
        if doctor_site_id == patient_site_id:
            query_times_same_site.append(query_time)
        else:
            query_times_diff_site.append(query_time)

avg_query_time_same_site = sum(query_times_same_site) / len(query_times_same_site)
avg_query_time_diff_site = sum(query_times_diff_site) / len(query_times_diff_site)

print(f"對於Site_ID和所欲訪問之Patient之Site_ID相同的doctor來說，平均query時間為: {avg_query_time_same_site} 秒")
print(f"對於Site_ID和所欲訪問之Patient之Site_ID不同的doctor來說，平均query時間為: {avg_query_time_diff_site} 秒")

cursor.close()
conn.close()

The 0st test.
